# Estudio de evolución temporal de aftershocks

Descripción del proyecto

## 1. Descargar los datos

link de google drive

In [ ]:
#https://drive.google.com/file/d/1Vh6umk8AlTCU5XEdpiRRjpy8N9gl3li0/view?usp=share_link

Para bajarlo se usa un link modificado

In [ ]:
#!gdown https://drive.google.com/uc?id=1Vh6umk8AlTCU5XEdpiRRjpy8N9gl3li0

In [ ]:
Descomprimir el archivo

In [ ]:
#unzip SPUD_bundle_2023-03-28T14.29.00.zip 

## 2. Importar paquetes

In [21]:
import numpy as np
import pandas as pd
#import fnmatch
import os
import glob
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
#import pygmt

## 3. Crear directorios

In [ ]:
!mkdir figs_num_time
!mkdir figs_mag_time
!mkdir figs_depth_time
!mkdir figs_comb

## 4. Leer y limpiar archivos de datos

In [22]:
#listar archivos
!rm SPUD_bundle_2023-03-28T14.29.00/After*/*copy.txt
dir_path="SPUD_bundle_2023-03-28T14.29.00"
list_files=[]
for file_dir in glob.glob(dir_path + "/**", recursive=True):
     if file_dir.endswith(".txt"):
         list_files.append(file_dir)

#print(list_files)

In [23]:
#leer archivos y limpiarlos
for tx_file in list_files:
    agencies=[",us",",US",",GCMT",",guc",",NEIC",";NEIC",",ISC",",IDC",",EIDC",",CASC",",CADCG",",SJS",",SSNC",",MEX",",IBQ",",IGQ",",TRN",",TAC",",SSS",",INET",",UVC",",RSMAC",",SCB",",GUC",",NEIS",",LIM",",SNET",",SDD",",UNM",",SJA","JB"]
    mags=["|mb,","|MW,","|Mww,","|mww,","|MWR,","|md,","|,","|ML,","|MB,","|MWW,","|MS,","|Mwr,","|mbtmp,","|MD,","|mw,","|mL,","|MSZ,","|MWB,","|mb_Lg,","|mb1mx,"]
    with open(tx_file,'r') as data:
        plaintext=data.read()
    for sust in agencies:
        plaintext=plaintext.replace(sust,"")
    for sust in mags:
        plaintext=plaintext.replace(sust,"|")
    with open(tx_file[:-4]+"_copy.txt", 'w') as f:
        f.write(plaintext)

## 4. Crear data frame y hacer gráficas de aftershocks

In [ ]:
#crear data frame para cada archivo
!rm figs_num_time/*
!rm figs_mag_time/*
!rm figs_depth_time/*
!rm figs_comb/*
p=[]
lat=[]
mag=[]
a_s=[]
#fec=[]
for tx_file in list_files:
    ex1='SPUD_bundle_2023-03-28T14.29.00/Aftershocks_9762033/1995.10.03.01.51.EventsList.txt'
    ex2='SPUD_bundle_2023-03-28T14.29.00/Aftershocks_9754820/2007.11.14.15.40.EventsList.txt'
    ex3='SPUD_bundle_2023-03-28T14.29.00/Aftershocks_11519080/2016.04.16.23.58.EventsList.txt'
    ex4='SPUD_bundle_2023-03-28T14.29.00/Aftershocks_9753523/1994.06.09.00.33.EventsList.txt'
    ex5='SPUD_bundle_2023-03-28T14.29.00/Aftershocks_9759503/1997.01.23.02.15.EventsList.txt'
    ex6='SPUD_bundle_2023-03-28T14.29.00/Aftershocks_9758029/2014.04.18.14.27.EventsList.txt'
    ex7='SPUD_bundle_2023-03-28T14.29.00/Aftershocks_9762691/2011.01.01.09.56.EventsList.txt'
    ex8='SPUD_bundle_2023-03-28T14.29.00/Aftershocks_9761373/1998.01.30.12.16.EventsList.txt'
    if (tx_file!=ex1) & (tx_file!=ex2) & (tx_file!=ex3) & (tx_file!=ex4) & (tx_file!=ex5) & (tx_file!=ex6) & (tx_file!=ex7) & (tx_file!=ex8):
        df = pd.read_csv(tx_file[:-4]+"_copy.txt", sep="|", header=None)
        df.columns =['Code', 'Date', 'Latitude', 'Longitude', 'Depth', 'catalog','catalog 2', 'catalog 3','magnitude', 'location']
        df[['fecha', 'tiempo']] = df['Date'].str.split(' ', 1, expand=True)
        df[['anio', 'mes', 'dia']] = df['fecha'].str.split('/', expand=True)
        df[['hora', 'minuto', 'segundo']] = df['tiempo'].str.split(':', expand=True)
        df.sort_values(by=['magnitude'], inplace=True, ascending=False)
        df=df.reset_index(drop=True)
        df['anio'] = df['anio'].astype('int')
        df['mes'] = df['mes'].astype('int')
        df['dia'] = df['dia'].astype('int')
        df['hora'] = df['hora'].astype('int')
        df['minuto'] = df['minuto'].astype('int')
        df['segundo'] = df['segundo'].astype('float64')
        
        #extraer información del mainshock
        main_year=df.loc[0,'anio']
        main_month=df.loc[0,'mes']
        main_day=df.loc[0,'dia']
        main_hour=df.loc[0,'hora']
        main_minute=df.loc[0,'minuto']
        main_sec=df.loc[0,'segundo']
        main_loc=df.loc[0,'location']
        main_mag=df.loc[0,'magnitude']
        main_lat=df.loc[0,'Latitude']
        main_date=df.loc[0,'fecha']
        d_bis=[0,31,60,91,121,152,182,213,244,274,305,335]
        d_norm=[0,31,59,90,120,151,181,212,243,273,304,334]
        nmes=[1,2,3,4,5,6,7,8,9,10,11,12]
        if (np.mod(main_year,4)==0):
            tyear=(main_year//4-1)*366+(main_year-main_year//4)*365
            tmonth=d_bis[nmes.index(main_month)]
        else:
            tyear=(main_year//4)*366+(main_year-1-main_year//4)*365
            tmonth=d_norm[nmes.index(main_month)]
        #tiempo del mainshock
        t0=main_sec + 60*main_minute + 3600*main_hour + 24*3600*(main_day-1) + tmonth*24*3600 + tyear*24*3600

        #calcular el tiempo de ocurrencia de los aftershocks después del mainshock
        df["t"]=np.nan
        #anio no bisiesto
        for ind in range(12):
            cond=(df['mes'] == nmes[ind]) & (np.mod(df['anio'],4) != 0)
            t_true=df["segundo"] + 60*df["minuto"] + 3600*df["hora"] + 24*3600*(df["dia"]-1) + d_norm[ind]*24*3600+((df.anio//4)*366+(df.anio-df.anio//4-1)*365)*24*3600 - t0
            df["t"]=np.where(cond,t_true,df["t"])
        #anio bisiesto
        for ind in range(12):
            cond=(df['mes'] == nmes[ind]) & (np.mod(df['anio'],4) == 0)
            t_true=df["segundo"] + 60*df["minuto"] + 3600*df["hora"] + 24*3600*(df["dia"]-1) + d_bis[ind]*24*3600+((df.anio//4-1)*366+(df.anio-df.anio//4)*365)*24*3600 - t0
            df['t']=np.where(cond,t_true,df['t'])
            
        #variación del número de aftershocks en el tiempo
        size=df.t.count()-1
        if (size>=50):
            print(size, main_loc, main_day, main_month, main_year, main_mag)
            interv=24*3600 #tamaño del intervalo (1 día)
            tmax=df['t'].max() #tiempo de la última réplica
            bot=0 #valor inicial del límite inferior del intervalo
            n=int(np.ceil(tmax / interv)) #número de intervalos
            t=[] #número de días
            a_shock=[] #cantidad de réplicas

            #llenado de las listas con el tiempo y el número de réplicas
            for num in range(1,n+1):
                top=interv*num
                check=df.t[(df.t>bot) & (df.t<=top)].count()
                if check > 0:
                    a_shock.append(df.t[(df.t>bot) & (df.t<=top)].count())
                    t.append(num)
                bot=top
    
            #definición de modelo de Omori para ajustar la curva
            def model_f(t,k,c,p):
                return k/(c+t)**p

            #ajuste de curva
            popt, pcov = curve_fit(model_f, t, a_shock)            
            k_opt, c_opt, p_opt = popt #parámetros del modelo
            
            #llenado de listas para análisis posterior de variaciones de
            p.append(p_opt)
            lat.append(main_lat)
            mag.append(main_mag)
            a_s.append(size)
            #fec.append(main_date)

            #puntos para el modelo
            x_model = np.linspace(min(t), max(t), 100)
            y_model = model_f(x_model, k_opt, c_opt, p_opt) 
 
            #gráfica de decaimiento
            plt.plot(t,a_shock,'o')
            plt.plot(x_model, y_model,'r')
            plt.xlim((0))
            plt.ylim((0))
            plt.xlabel("Días después del terremoto")
            plt.ylabel("Número de réplicas")
            title="M"+str(main_mag)+" EARTHQUAKE "+main_loc+", "+str(int(main_day))+"-"+str(int(main_month))+"-"+str(int(main_year))+" ("+str(size)+" réplicas)"
            plt.title(title)
            plt.text(4.5, max(a_shock)-max(a_shock)/5,
                     'k='+str(round(k_opt,2))+', c='+str(round(c_opt,2))+', p='+str(round(p_opt,2)), 
                     horizontalalignment='left',verticalalignment='bottom'
                    )
            plt.savefig('figs_num_time/num_time_'+main_loc+", "+str(int(main_day))+"-"+str(int(main_month))+"-"+str(int(main_year)),bbox_inches='tight')
            plt.show()

        #grafica de magnitud vs tiempo
            plt.figure(figsize=(7.6,4))
            plt.scatter(df.t[(df.t>=0)]/interv,df.magnitude[(df.t>=0)],s=df.magnitude[(df.t>=0)]*30,c=df.Depth[(df.t>=0)],alpha=0.5)
            plt.xlabel("Días después del terremoto")
            plt.ylabel("Magnitud de las réplicas")
            plt.colorbar(label='profundidad (km)')
            plt.title(title)
            plt.savefig('figs_mag_time/mag_time_'+main_loc+", "+str(int(main_day))+"-"+str(int(main_month))+"-"+str(int(main_year)),bbox_inches='tight')
            plt.show()
        
        #grafica combinada
            fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15, 4),gridspec_kw={'width_ratios': [8,10],'height_ratios':[1]})
            fig.suptitle(title)
            ax1.plot(t,a_shock,'o')
            ax1.plot(x_model, y_model,'r')
            ax1.set_xlim((0))
            ax1.set_ylim((0))
            ax1.set_xlabel("Días después del terremoto")
            ax1.set_ylabel("Número de réplicas")
            ax1.text(4.5, max(a_shock)-max(a_shock)/5,
                     'k='+str(round(k_opt,2))+', c='+str(round(c_opt,2))+', p='+str(round(p_opt,2)), 
                     horizontalalignment='left',verticalalignment='bottom'
                    )
            pcm=ax2.scatter(df.t[(df.t>=0)]/interv,df.magnitude[(df.t>=0)],s=df.magnitude[(df.t>=0)]*30,c=df.Depth[(df.t>=0)],alpha=0.5)
            ax2.set_xlabel("Días después del terremoto")
            ax2.set_ylabel("Magnitud de las réplicas")
            ax=ax2
            fig.colorbar(pcm,ax=ax2,label='profundidad (km)')
            plt.savefig('figs_comb/comb_'+main_loc+", "+str(int(main_day))+"-"+str(int(main_month))+"-"+str(int(main_year)),bbox_inches='tight')
            plt.show()
            
        #grafica de profundidad vs tiempo
            plt.figure(figsize=(7.6,4))
            plt.scatter(df.t[(df.t>=0)]/interv,df.Depth[(df.t>=0)],s=df.magnitude[(df.t>=0)]*15,c=df.magnitude[(df.t>=0)],alpha=0.5)
            plt.xlabel("Días después del terremoto")
            plt.ylabel("Profundidad de las réplicas")
            plt.colorbar(label='magnitud')
            plt.title("M"+str(main_mag)+" EARTHQUAKE "+main_loc+", "+str(int(main_day))+"-"+str(int(main_month))+"-"+str(int(main_year))+" ("+str(size)+" réplicas)")
            plt.savefig('figs_depth_time/depth_time_'+main_loc+", "+str(int(main_day))+"-"+str(int(main_month))+"-"+str(int(main_year)),bbox_inches='tight')
            plt.show()
    
            

In [ ]:
#Variaciones de p
#eliminar eventos repetidos con menos replicas [8 10 13 19]
for ind in [8, 9, 11, 16]:
    del p[ind]
    del lat[9]
    del a_s[11]
    del mag[11]
#gráficas
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15, 4))
fig.suptitle("Variación de la constante de decaimiento p")
ax1.plot(llim,plim_u,'k--')
ax1.plot(llim,plim_d,'k--')
pcm=ax1.scatter(lat, p,s=70,c=a_s,alpha=0.5)
fig.colorbar(pcm,ax=ax1,label='Número de réplicas')
ax1.set_xlabel("Latitud")
ax1.set_ylabel("p")
ax2.plot(mlim,plim_u,'k--')
ax2.plot(mlim,plim_d,'k--')
pcm=ax2.scatter(mag, p,s=70,c=a_s,alpha=0.5)
ax2.set_xlabel("Magnitud")
ax2.set_ylabel("p")
fig.colorbar(pcm,ax=ax2,label='Número de réplicas')
plt.savefig('p_comp',bbox_inches='tight')
plt.show()


In [ ]:
#selected earthquakes
print(main_lon,main_lat,main_depth,main_mag, main_date, main_loc)
